In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v0 import CNN_v0

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
cnn = CNN_v0((480, 640, 3), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v0/weights/"

In [7]:
model_names = ["cnn_v0_1e_1_100",
               "cnn_v0_1e_3_10",
               "cnn_v0_1e_7_5",
               "cnn_v0_1e_3_50_2",
               "cnn_v0_1e_5_150_2",
               "cnn_v0_5e_6_20_2",
               "cnn_v0_1e_7_31_2"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v0((480, 640, 3), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 646ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 763ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 855ms/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.6931 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.6931 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v0/weights/cnn_v0_1e_1_100 behaved:
For cut-of-point 0.2 had accuracy 86.42958247759832:
[ -1   5  -5   4   5   5   4   2   8   5   8   6  -3   7   8   6   7  -1
  -1   4  -3   8   0   1   0  -1  -4   1   6   4   6   2  -5   5   9   2
   9   8   7   2   5   8  14   1   7   9  10   0   3   3  10   4   1  -1
  -3   2  -4  -3   2   1  -3 -12 -10  -5   0   4 -10  -2   0   0   3   2
   2   4   4   2  -3   3  -3   4   0   4   4   1   0   2   6   0   8   4
   4  -1   2  -7   0 -17  -7  -3  -4  -1   8  -4   3   8   8   6   7  -2
  -5   2   3   6   3  -1   2  12   9   0   6   7  -3   3   1   2   3  -1
   0   0  -1   8   6   2   2   6   1   2   5   1  10   3  10   2   4  -4
   2   2  -3  13  -2   6  -1   1   3  -2   2   4   2  -3   6   0   1   9
  -1  -1  -6  -8   1  -2   2   7  -1   3   2   1   3   0   3   0   3   6
   3   4   1   1   2   3   1   4   8  10   2   2   0   0   0  10   5   8
   9   4   9  10   8   8   2   3   2  -4   1  -5  -5  -4  -5  -2  -4   3
   4   9   3  -4   3 